<a href="https://colab.research.google.com/github/Alexxxa016/AI/blob/main/SearchTreeCheckrs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SearchTree checkers Endgame

### Description of Game

This is an endgame scenario for checkers on an 8×8 board. The program uses a CurrentBoard class to manage the board state and move history, and it includes an all_moves function to generate all legal moves. A search tree is built using a minimax algorithm with optional alpha‑beta pruning guided by a heuristic evaluation function that factors in material, mobility, and positioning. The game runs computer‑vs‑computer.

# Pre- current board code
###(attempts of board creating, parsing, and imports)

In [3]:
#########################################################################
              # Imports and Defining Bounds
#########################################################################
import copy

def in_bounds(r, c):
    return 0 <= r < 8 and 0 <= c < 8

def deep_copy_board(board):
    return copy.deepcopy(board)

In [5]:
#########################################################################
              # Empty board visualisation
#########################################################################

def empty_board():
    board = []
    for row in range(8):
        board.append(["."] * 8)
    return board

def print_board(board):
    # Column labels a-h
    col_labels = "   " + " ".join(chr(c) for c in range(ord('a'), ord('h')+1))
    print(col_labels)
    for row in range(8):
        print(f"{row}  " + " ".join(board[row]))
    print()

board = empty_board()
print_board(board)


   a b c d e f g h
0  . . . . . . . .
1  . . . . . . . .
2  . . . . . . . .
3  . . . . . . . .
4  . . . . . . . .
5  . . . . . . . .
6  . . . . . . . .
7  . . . . . . . .



In [6]:
#########################################################################
     # Empty board but (".") are dark and (" ")are light squares
#########################################################################

def checkers_board():
    board = []
    for row in range(8):
        row_list = []
        for col in range(8):
            if (row + col) % 2 == 1:
                row_list.append(".")
            else:
                row_list.append(" ")
        board.append(row_list)
    return board
#8x8 checkers board with playable dark squares (".") and non-playable light squares (" ").
def print_board(board):
    col_labels = "   " + " ".join(chr(c) for c in range(ord('a'), ord('h')+1))
    print(col_labels)
    for row in range(8):
        print(f"{row}  " + " ".join(board[row]))
    print()

board = checkers_board()
print_board(board)


   a b c d e f g h
0    .   .   .   .
1  .   .   .   .  
2    .   .   .   .
3  .   .   .   .  
4    .   .   .   .
5  .   .   .   .  
6    .   .   .   .
7  .   .   .   .  



In [7]:
#########################################################################
        #Parsing positions for easier navigation through board
#########################################################################


# Convert a string like "a3" to board coordinates (row, col)
def parse_position(pos):

  pos = pos.strip().lower()
  rowpos = ""
  colpos = ""

  for char in pos:
    if char.isdigit():
      rowpos += char
    elif char.isalpha():
      colpos += char
  if rowpos == "" or colpos == "":
    raise ValueError("Invalid position format")
  row = int(rowpos)
  col = ord(colpos[0]) - ord('a')
  return row, col




In [8]:
print(parse_position("2f"))  #(2, 5)
print(parse_position("4c"))  #(4, 2)
print(parse_position("6h"))  #(6, 7)
print(parse_position("4d"))  #(4, 3)
print(parse_position("0a"))  #(0, 0)
print(parse_position("7b"))  #(7, 1)

(2, 5)
(4, 2)
(6, 7)
(4, 3)
(0, 0)
(7, 1)


#CurrentBoard


This code sets up an endgame scenario for checkers using an 8×8 board. It creates a CurrentBoard class that initializes the board (with dark squares as playable areas), displays it, and tracks move history to avoid repeats. The class includes functions to generate all legal moves for each piece—both normal moves and captures

In [15]:
#########################################################################
                       # CurrentBoard Class
#########################################################################


#board state and history of moves to track repeating positions
class CurrentBoard:
    def __init__(self, board=None, history=None):
        if board is None:
            self.board = self.end_game_board()
        else:
            self.board = board
        if history is None:
            self.history = []
        else:
            self.history = history
#endgame board setup
    def end_game_board(self):
        board = []
        for row in range(8):
            row_list = []
            for col in range(8):
                if (row + col) % 2 == 1:
                    row_list.append(".")
                else:
                    row_list.append(" ")
            board.append(row_list)
        board[5][4] = "b"
        board[3][4] = "B"
        board[3][0] = "W"
        #board[3][0] = "W"
        #board[2][5] = "b"
        #board[4][3] = "B"
        return board

    def display(self):
        col_labels = "   " + " ".join(chr(c) for c in range(ord('a'), ord('h') + 1))
        print(col_labels)
        for row in range(8):
            print(f"{row}  " + " ".join(self.board[row]))
        print()

    def state_of_board(self):
        white_moves = len(self.all_moves("w"))
        black_moves = len(self.all_moves("b"))
        if white_moves == 0:
            return "black"
        if black_moves == 0:
            return "white"
        return "Unfinished"

    def opponent(self, piece):
        return "b" if piece.lower() == "w" else "w"
#piece movement
    def piece_moves(self, piece):
        if piece in ("B", "W"):
            return [(-1, -1), (-1, 1), (1, -1), (1, 1)]
        elif piece == "b":
            return [(1, -1), (1, 1)]
        elif piece == "w":
            return [(-1, -1), (-1, 1)]
        return []


        # This function generates all legal moves for the piece at position (r, c).
    # It checks normal moves and capture moves. If a capture is possible, that move is prioritized
    def generate_moves(self, r, c):
        piece = self.board[r][c]
        if piece not in ("b", "B", "w", "W"):
            return []
        moves = []
        captures = []
        directions = self.piece_moves(piece)
        opp = self.opponent(piece)
        for dr, dc in directions:
            new_r, new_c = r + dr, c + dc
            if in_bounds(new_r, new_c):
                if self.board[new_r][new_c] == ".":
                    moves.append((new_r, new_c, None))
                else:
                    target = self.board[new_r][new_c]
                    if target.lower() == opp:
                        cap_r, cap_c = r + 2*dr, c + 2*dc
                        if in_bounds(cap_r, cap_c) and self.board[cap_r][cap_c] == ".":
                            captures.append((cap_r, cap_c, (new_r, new_c)))
        return captures if captures else moves
    CurrentBoard.generate_moves = generate_moves
        #all possible moves
        #produce list
        # returns a list of new board states resulting from each move.
    def all_moves(self, player):
        capture_moves = []
        non_capture_moves = []
        for r in range(8):
            for c in range(8):
                if self.board[r][c].lower() == player:
                    moves = self.generate_moves(r, c)
                    for move in moves:
                        if move[2] is not None:
                            capture_moves.append(((r, c), move))
                        else:
                            non_capture_moves.append(((r, c), move))
                            #mandatory capture rule
        moves = capture_moves if capture_moves else non_capture_moves
        boards = []
        for (r, c), move in moves:
            new_board = deep_copy_board(self.board)
            piece = new_board[r][c]
            new_board[r][c] = "."
            dest_r, dest_c, capture = move
            # Promote pawn to king if it reaches the last row.
            if capture is not None:
                cap_r, cap_c = capture
                new_board[cap_r][cap_c] = "."
            if piece == "b" and dest_r == 7:
                piece = "B"
            elif piece == "w" and dest_r == 0:
                piece = "W"
            new_board[dest_r][dest_c] = piece
            new_history = self.history + [self.board]
            boards.append(CurrentBoard(new_board, history=new_history))
        return boards



In [16]:
#########################################################################
              # Displaying the board setup im going with
#########################################################################

cb = CurrentBoard()
cb.display()

   a b c d e f g h
0    .   .   .   .
1  .   .   .   .  
2    .   .   .   .
3  W   .   B   .  
4    .   .   .   .
5  .   .   b   .  
6    .   .   .   .
7  .   .   .   .  



In [17]:
#########################################################################
              # All legal moves for Black
#########################################################################
cb = CurrentBoard()
cb.display()
print("Legal moves for Black:")
for new_state in cb.all_moves("b"):
    new_state.display()



   a b c d e f g h
0    .   .   .   .
1  .   .   .   .  
2    .   .   .   .
3  W   .   B   .  
4    .   .   .   .
5  .   .   b   .  
6    .   .   .   .
7  .   .   .   .  

Legal moves for Black:
   a b c d e f g h
0    .   .   .   .
1  .   .   .   .  
2    .   B   .   .
3  W   .   .   .  
4    .   .   .   .
5  .   .   b   .  
6    .   .   .   .
7  .   .   .   .  

   a b c d e f g h
0    .   .   .   .
1  .   .   .   .  
2    .   .   B   .
3  W   .   .   .  
4    .   .   .   .
5  .   .   b   .  
6    .   .   .   .
7  .   .   .   .  

   a b c d e f g h
0    .   .   .   .
1  .   .   .   .  
2    .   .   .   .
3  W   .   .   .  
4    .   B   .   .
5  .   .   b   .  
6    .   .   .   .
7  .   .   .   .  

   a b c d e f g h
0    .   .   .   .
1  .   .   .   .  
2    .   .   .   .
3  W   .   .   .  
4    .   .   B   .
5  .   .   b   .  
6    .   .   .   .
7  .   .   .   .  

   a b c d e f g h
0    .   .   .   .
1  .   .   .   .  
2    .   .   .   .
3  W   .   B   .  
4    .   .   .   .
5  

In [18]:
#########################################################################
              # All legal moves for white
#########################################################################
cb = CurrentBoard()
cb.display()
print("Legal moves for White:")
for new_state in cb.all_moves("w"):
    new_state.display()


   a b c d e f g h
0    .   .   .   .
1  .   .   .   .  
2    .   .   .   .
3  W   .   B   .  
4    .   .   .   .
5  .   .   b   .  
6    .   .   .   .
7  .   .   .   .  

Legal moves for White:
   a b c d e f g h
0    .   .   .   .
1  .   .   .   .  
2    W   .   .   .
3  .   .   B   .  
4    .   .   .   .
5  .   .   b   .  
6    .   .   .   .
7  .   .   .   .  

   a b c d e f g h
0    .   .   .   .
1  .   .   .   .  
2    .   .   .   .
3  .   .   B   .  
4    W   .   .   .
5  .   .   b   .  
6    .   .   .   .
7  .   .   .   .  



#Evaluation function

###Evaluation Evolution

In [126]:
#########################################################################
                       # Initial Evaluation stage
#########################################################################
'''
This stage looks at the board and adds up points based on your pieces and their positions. It gives extra points for:

Having strong pieces.
Controlling the center of the board.
Having more possible moves.
Having moves that force the opponent to capture.

It basically tells you if your board looks good based on material and position.

'''



def evaluate_stage1(self):
    # Stage 1: Basic evaluation
    base_score = 0      # values of pieces
    mobility_white = 0  # total num of legal moves for White
    mobility_black = 0  # total num of legal moves for Black
    static_center_bonus = 0  # bonus for central squares
    forced_capture_bonus = 0  # bonus for forced captures
    white_forced_capture = False
    black_forced_capture = False
    center_squares = {(3, 3), (3, 4), (4, 3), (4, 4)}

    for r in range(8):
        for c in range(8):
            cell = self.board[r][c]
            if cell == "W":
                base_score += 2
                if (r, c) in center_squares:
                    static_center_bonus += 0.5
                moves = self.generate_moves(r, c)
                mobility_white += len(moves)
                if any(move[2] is not None for move in moves):
                    white_forced_capture = True
            elif cell == "w":
                base_score += 1
                if (r, c) in center_squares:
                    static_center_bonus += 0.3
                moves = self.generate_moves(r, c)
                mobility_white += len(moves)
                if any(move[2] is not None for move in moves):
                    white_forced_capture = True
            elif cell == "B":
                base_score -= 2
                if (r, c) in center_squares:
                    static_center_bonus -= 0.5
                moves = self.generate_moves(r, c)
                mobility_black += len(moves)
                if any(move[2] is not None for move in moves):
                    black_forced_capture = True
            elif cell == "b":
                base_score -= 1
                if (r, c) in center_squares:
                    static_center_bonus -= 0.3
                moves = self.generate_moves(r, c)
                mobility_black += len(moves)
                if any(move[2] is not None for move in moves):
                    black_forced_capture = True

    mobility_score = 0.1 * (mobility_white - mobility_black)
    if white_forced_capture:
        forced_capture_bonus += 0.5
    if black_forced_capture:
        forced_capture_bonus -= 0.5
    total_score = base_score + static_center_bonus + mobility_score + forced_capture_bonus

    print("Stage 1: Basic Evaluation Score =", total_score)
    return total_score

CurrentBoard.evaluate_stage1 = evaluate_stage1



In [127]:
board_instance = CurrentBoard()
board_instance.evaluate_stage1()

Stage 1: Basic Evaluation Score = -1.9


-1.9

In [128]:
#########################################################################
                       # Evaluation stage 2
#########################################################################
'''
Starting with the score from Stage 1, this stage checks if the same board position has happened before. It subtracts points if:

The board repeats positions.
The same moves happen over and over.

This helps discourage making the same moves repeatedly and encourages trying new strategies.
'''


def evaluate_stage2(self):
    # Start with the result from Stage 1.
    total_score = self.evaluate_stage1()

    repetition_penalty = 0
    for hist_board in self.history:
        if self.board == hist_board:
            repetition_penalty -= 200
            break
    if self.history and self.board == self.history[-1]:
        repetition_penalty -= 500
    rep_count = sum(1 for hist_board in self.history if hist_board == self.board)
    if rep_count > 1:
        repetition_penalty -= 300 * (rep_count - 1)
    if self.history:
        prev_white = sum(cell in ("w", "W") for row in self.history[-1] for cell in row)
        prev_black = sum(cell in ("b", "B") for row in self.history[-1] for cell in row)
        cur_white = sum(cell in ("w", "W") for row in self.board for cell in row)
        cur_black = sum(cell in ("b", "B") for row in self.board for cell in row)
        if cur_white == prev_white and cur_black == prev_black:
            repetition_penalty -= 100

    total_score += repetition_penalty
    print("Stage 2: After Repetition Penalties, Score =", total_score)
    return total_score


CurrentBoard.evaluate_stage2 = evaluate_stage2


In [129]:
board_instance = CurrentBoard()
board_instance.evaluate_stage2()

Stage 1: Basic Evaluation Score = -1.9
Stage 2: After Repetition Penalties, Score = -1.9


-1.9

In [130]:
#########################################################################
                       # Evaluation stage 3
#########################################################################
'''
A bonus is added when the number of opponent pieces captured or
the preservation of friendly pieces has improved relative to the
previous board state.

'''


def evaluate_stage3(self):
    # Start with the result from Stage 2.
    total_score = self.evaluate_stage2()

    improvement_bonus = 0
    if self.history:
        prev_board = self.history[-1]
        prev_white = sum(cell in ("w", "W") for row in prev_board for cell in row)
        prev_black = sum(cell in ("b", "B") for row in prev_board for cell in row)
        cur_white = sum(cell in ("w", "W") for row in self.board for cell in row)
        cur_black = sum(cell in ("b", "B") for row in self.board for cell in row)
        improvement_bonus += ((prev_black - cur_black) - (prev_white - cur_white)) * 15

    total_score += improvement_bonus
    print("Stage 3: After Material Improvement Bonus, Score =", total_score)
    return total_score
CurrentBoard.evaluate_stage3 = evaluate_stage3


In [131]:
board_instance = CurrentBoard()
board_instance.evaluate_stage3()

Stage 1: Basic Evaluation Score = -1.9
Stage 2: After Repetition Penalties, Score = -1.9
Stage 3: After Material Improvement Bonus, Score = -1.9


-1.9

In [132]:
#########################################################################
                       # Evaluation stage 4
#########################################################################
'''
This stage calculates a bonus based on the proximity
 of pieces to the center of the board.

This helps to steer pieces away from corners and edges,
which generally leads to better mobility and overall board influence.

'''
def evaluate_stage4(self):
    # Start with the result from Stage 3.
    total_score = self.evaluate_stage3()

    dynamic_center_bonus = 0
    center_x, center_y = 3.5, 3.5
    for r in range(8):
        for c in range(8):
            if self.board[r][c] in ("w", "W"):
                dist = abs(r - center_x) + abs(c - center_y)
                dynamic_center_bonus += max(0, 5 - dist) * 0.2
            elif self.board[r][c] in ("b", "B"):
                dist = abs(r - center_x) + abs(c - center_y)
                dynamic_center_bonus -= max(0, 5 - dist) * 0.2

    total_score += dynamic_center_bonus
    print("Stage 4: After Dynamic Center Bonus, Score =", total_score)
    return total_score
CurrentBoard.evaluate_stage4 = evaluate_stage4

In [133]:
board_instance = CurrentBoard()
board_instance.evaluate_stage4()

Stage 1: Basic Evaluation Score = -1.9
Stage 2: After Repetition Penalties, Score = -1.9
Stage 3: After Material Improvement Bonus, Score = -1.9
Stage 4: After Dynamic Center Bonus, Score = -3.1


-3.1

In [134]:
#########################################################################
                       # Evaluation stage 5
#########################################################################
'''
This stage rewards forward movement by awarding extra points for pieces
that advance toward the opponent’s side.
Additional points are given for positioning pieces near promotion zones,
encouraging aggressive and progressive play,
leading to increased offensive potential and chances for piece promotion.

'''

def evaluate_stage5(self):
    # Start with the result from Stage 4.
    total_score = self.evaluate_stage4()
    advancement_bonus = 0
    for r in range(8):
        for c in range(8):
            if self.board[r][c] == "w":
                advancement_bonus += (7 - r) * 0.5
            elif self.board[r][c] == "b":
                advancement_bonus += r * 0.5
    promotion_bonus = 0
    for c in range(8):
        if self.board[1][c] == "w":
            promotion_bonus += 2
        if self.board[6][c] == "b":
            promotion_bonus += 2
    bonus = advancement_bonus + promotion_bonus
    total_score += bonus
    print("Stage 5: After Advancement & Promotion Bonuses, Score =", total_score)
    return total_score
CurrentBoard.evaluate_stage5 = evaluate_stage5

In [135]:
board_instance = CurrentBoard()
board_instance.evaluate_stage5()

Stage 1: Basic Evaluation Score = -1.9
Stage 2: After Repetition Penalties, Score = -1.9
Stage 3: After Material Improvement Bonus, Score = -1.9
Stage 4: After Dynamic Center Bonus, Score = -3.1
Stage 5: After Advancement & Promotion Bonuses, Score = -0.6000000000000001


-0.6000000000000001

In [136]:
#########################################################################
                       # Evaluation stage 6
#########################################################################
'''
When Black’s pawn structure is active and centrally positioned,
the evaluation shifts in Black's favour. This was implemented
because the black pawn was not used throughout the game.

'''

def evaluate_stage6(self):
    # Start with the result from Stage 5.
    total_score = self.evaluate_stage5()

    # Pawn Activity Bonus for Black (should reduce score if Black benefits)
    pawn_activity_bonus = 0
    for r in range(8):
        for c in range(8):
            if self.board[r][c] == "b":
                pawn_activity_bonus -= 5  # Subtracting favors Black
    total_score += pawn_activity_bonus
    print("Stage 6a: After Pawn Activity Bonus, Score =", total_score)

    # Pawn Center Bonus for Black (should also reduce score if Black benefits)
    pawn_center_bonus = 0
    center_x, center_y = 3.5, 3.5
    for r in range(8):
        for c in range(8):
            if self.board[r][c] == "b":
                dist = abs(r - center_x) + abs(c - center_y)
                pawn_center_bonus -= max(0, 7 - dist) * 2  # Subtracting favors Black
    total_score += pawn_center_bonus
    print("Stage 6b: After Pawn Center Bonus, Score =", total_score)

    return total_score

CurrentBoard.evaluate_stage6 = evaluate_stage6


In [137]:
board_instance = CurrentBoard()
board_instance.evaluate_stage6()

Stage 1: Basic Evaluation Score = -1.9
Stage 2: After Repetition Penalties, Score = -1.9
Stage 3: After Material Improvement Bonus, Score = -1.9
Stage 4: After Dynamic Center Bonus, Score = -3.1
Stage 5: After Advancement & Promotion Bonuses, Score = -0.6000000000000001
Stage 6a: After Pawn Activity Bonus, Score = -5.6
Stage 6b: After Pawn Center Bonus, Score = -15.6


-15.6

In [138]:
#########################################################################
                       # Evaluation stage 7
#########################################################################
'''
Applies a penalty for pieces on the board's edge, where mobility is limited.
Subtracting points here lowers the score, indicating a less effective
position. This stage benefits White when the score remains higher
(with fewer pieces on the edge) and favors Black when the score is more
negative, reflecting overall board control and piece mobility.

'''

def evaluate_stage7(self):
    # Start with the result from Stage 6.
    total_score = self.evaluate_stage6()
    edge_penalty = 0
    for r in range(8):
        for c in range(8):
            if r in (0, 7) or c in (0, 7):
                if self.board[r][c] in ("w", "W", "b", "B"):
                    edge_penalty -= 5
    total_score += edge_penalty
    print("Stage 7: After Edge Penalty, Score =", total_score)
    return total_score

CurrentBoard.evaluate_stage7 = evaluate_stage7

In [139]:
board_instance = CurrentBoard()
board_instance.evaluate_stage7()

Stage 1: Basic Evaluation Score = -1.9
Stage 2: After Repetition Penalties, Score = -1.9
Stage 3: After Material Improvement Bonus, Score = -1.9
Stage 4: After Dynamic Center Bonus, Score = -3.1
Stage 5: After Advancement & Promotion Bonuses, Score = -0.6000000000000001
Stage 6a: After Pawn Activity Bonus, Score = -5.6
Stage 6b: After Pawn Center Bonus, Score = -15.6
Stage 7: After Edge Penalty, Score = -20.6


-20.6

In [140]:
#########################################################################
                       # Evaluation stage 8
#########################################################################
'''
When White's pieces have available capturing moves, a bonus of 50 points
is added for each capture opportunity, while Black's capturing
moves subtract 30 points. This shifts the evaluation in favor of
the side with more aggressive capture opportunities. The implementation
was chosen to reward immediate offensive threats that can lead to material
gain.

'''

def evaluate_stage8(self):
    # Start with the result from Stage 7.
    total_score = self.evaluate_stage7()
    capture_bonus = 0
    for r in range(8):
        for c in range(8):
            piece = self.board[r][c]
            moves = self.generate_moves(r, c)
            if piece.lower() == "w":
                for move in moves:
                    if move[2] is not None:
                        capture_bonus += 50
            elif piece.lower() == "b":
                for move in moves:
                    if move[2] is not None:
                        capture_bonus -= 30
    total_score += capture_bonus
    print("Stage 8: After Aggressive Capture Bonus, Score =", total_score)
    return total_score

CurrentBoard.evaluate_stage8 = evaluate_stage8

In [141]:
board_instance = CurrentBoard()
board_instance.evaluate_stage8()

Stage 1: Basic Evaluation Score = -1.9
Stage 2: After Repetition Penalties, Score = -1.9
Stage 3: After Material Improvement Bonus, Score = -1.9
Stage 4: After Dynamic Center Bonus, Score = -3.1
Stage 5: After Advancement & Promotion Bonuses, Score = -0.6000000000000001
Stage 6a: After Pawn Activity Bonus, Score = -5.6
Stage 6b: After Pawn Center Bonus, Score = -15.6
Stage 7: After Edge Penalty, Score = -20.6
Stage 8: After Aggressive Capture Bonus, Score = -20.6


-20.6

In [142]:
#########################################################################
                       # Evaluation stage 9
#########################################################################
'''
This was implemented to stop ping-pong movement on blacks side

'''

def evaluate_stage9(self):
    # Start with the result from Stage 8.
    total_score = self.evaluate_stage8()
    repetition_penalty_black = 0
    if self.history:
        prev_board = self.history[-1]
        for r in range(8):
            for c in range(8):
                if self.board[r][c] == "B" and prev_board[r][c] == "B":
                    repetition_penalty_black -= 100
    total_score += repetition_penalty_black
    print("Stage 9: After Black King Repetition Penalty, Score =", total_score)
    return total_score

CurrentBoard.evaluate_stage9 = evaluate_stage9

In [143]:
board_instance = CurrentBoard()
board_instance.evaluate_stage9()

Stage 1: Basic Evaluation Score = -1.9
Stage 2: After Repetition Penalties, Score = -1.9
Stage 3: After Material Improvement Bonus, Score = -1.9
Stage 4: After Dynamic Center Bonus, Score = -3.1
Stage 5: After Advancement & Promotion Bonuses, Score = -0.6000000000000001
Stage 6a: After Pawn Activity Bonus, Score = -5.6
Stage 6b: After Pawn Center Bonus, Score = -15.6
Stage 7: After Edge Penalty, Score = -20.6
Stage 8: After Aggressive Capture Bonus, Score = -20.6
Stage 9: After Black King Repetition Penalty, Score = -20.6


-20.6

In [144]:
#########################################################################
                       # Evaluation stage 10
#########################################################################
'''
This stage checks if either side has lost all pieces. If White has no
pieces remaining, a score of –1000 is returned; if Black has no pieces,
a score of +1000 is returned. This decisive adjustment immediately
recognizes a win or loss situation, reflecting the ultimate benefit
when one side is completely eliminated.

'''

def evaluate_stage10(self):
    # Start with the result from Stage 9.
    total_score = self.evaluate_stage9()
    cur_white = sum(cell in ("w", "W") for row in self.board for cell in row)
    cur_black = sum(cell in ("b", "B") for row in self.board for cell in row)
    if cur_white == 0:
        print("Stage 10: Terminal state reached (White has no pieces).")
        return -1000
    if cur_black == 0:
        print("Stage 10: Terminal state reached (Black has no pieces).")
        return 1000
    print("Stage 10: No terminal state reached, Score =", total_score)
    return total_score

CurrentBoard.evaluate_stage10 = evaluate_stage10

In [148]:

board_instance = CurrentBoard()
board_instance.evaluate_stage10()

Stage 1: Basic Evaluation Score = -1.9
Stage 2: After Repetition Penalties, Score = -1.9
Stage 3: After Material Improvement Bonus, Score = -1.9
Stage 4: After Dynamic Center Bonus, Score = -3.1
Stage 5: After Advancement & Promotion Bonuses, Score = -0.6000000000000001
Stage 6a: After Pawn Activity Bonus, Score = -5.6
Stage 6b: After Pawn Center Bonus, Score = -15.6
Stage 7: After Edge Penalty, Score = -20.6
Stage 8: After Aggressive Capture Bonus, Score = -20.6
Stage 9: After Black King Repetition Penalty, Score = -20.6
Stage 10: No terminal state reached, Score = -20.6


-20.6

In [149]:
#########################################################################
                       # Evaluation stage 11
#########################################################################
'''
When there are 6 or fewer pieces on the board, the current score is
amplified by 50% to emphasize the importance of the endgame. Additionally,
the distance between the kings is taken into account—if the evaluation
already favors one side, a bonus or further subtraction based on closer
king proximity is applied. This was implemented to stress the strategic
value of king activity and proximity during the endgame phase.

'''

def evaluate_stage11(self):
    # Start with the result from Stage 10.
    total_score = self.evaluate_stage10()
    total_pieces = sum(cell in ("w", "W", "b", "B") for row in self.board for cell in row)
    endgame_bonus = 0
    if total_pieces <= 6:
        # For demonstration, amplify the current score.
        endgame_bonus += 0.5 * total_score
        white_king_positions = [(r, c) for r in range(8) for c in range(8) if self.board[r][c] == "W"]
        black_king_positions = [(r, c) for r in range(8) for c in range(8) if self.board[r][c] == "B"]
        if white_king_positions and black_king_positions:
            wk = white_king_positions[0]
            bk = black_king_positions[0]
            distance = abs(wk[0] - bk[0]) + abs(wk[1] - bk[1])
            if total_score > 0:
                endgame_bonus += (7 - distance) * 20
            elif total_score < 0:
                endgame_bonus -= (7 - distance) * 20
    total_score += endgame_bonus
    print("Stage 11: After Endgame Bonus, Final Evaluation Score =", total_score)
    return total_score

CurrentBoard.evaluate_stage11 = evaluate_stage11

In [150]:
'''
the final score of –90.9 indicates a strong advantage for Black,
with the most significant shifts coming from active Black pawn play
and favorable endgame conditions.

This does not mean that black will win, its simply shows that,
based on the evaluation criteria, the current board position has
several elements that favor Black. The evaluation function is providing
a snapshot of the position by weighing factors like piece activity,
central control, and potential endgame strength, but it is not a guarantee
of the final outcome.
'''

board_instance = CurrentBoard()
board_instance.evaluate_stage11()

Stage 1: Basic Evaluation Score = -1.9
Stage 2: After Repetition Penalties, Score = -1.9
Stage 3: After Material Improvement Bonus, Score = -1.9
Stage 4: After Dynamic Center Bonus, Score = -3.1
Stage 5: After Advancement & Promotion Bonuses, Score = -0.6000000000000001
Stage 6a: After Pawn Activity Bonus, Score = -5.6
Stage 6b: After Pawn Center Bonus, Score = -15.6
Stage 7: After Edge Penalty, Score = -20.6
Stage 8: After Aggressive Capture Bonus, Score = -20.6
Stage 9: After Black King Repetition Penalty, Score = -20.6
Stage 10: No terminal state reached, Score = -20.6
Stage 11: After Endgame Bonus, Final Evaluation Score = -90.9


-90.9

###Complete Evaluation function

In [154]:
#########################################################################
                       # Complete Evaluation function
#########################################################################


def evaluate(self):
    ''' Base Evaluation (Material, Mobility, Static Center) '''
    base_score = 0  # values of pieces
    mobility_white = 0  # total num of legal moves
    mobility_black = 0
    static_center_bonus = 0  # reward more central pieces
    forced_capture_bonus = 0  # encourages positions where pieces are forced to capture
    white_forced_capture = False
    black_forced_capture = False
    center_squares = {(3, 3), (3, 4), (4, 3), (4, 4)}

    # Material, center, and mobility evaluation
    for r in range(8):
        for c in range(8):
            cell = self.board[r][c]
            if cell == "W":
                base_score += 2
                if (r, c) in center_squares:
                    static_center_bonus += 0.5
                moves = self.generate_moves(r, c)
                mobility_white += len(moves)
                if any(move[2] is not None for move in moves):
                    white_forced_capture = True
            elif cell == "w":
                base_score += 1
                if (r, c) in center_squares:
                    static_center_bonus += 0.3
                moves = self.generate_moves(r, c)
                mobility_white += len(moves)
                if any(move[2] is not None for move in moves):
                    white_forced_capture = True
            elif cell == "B":
                base_score -= 2
                if (r, c) in center_squares:
                    static_center_bonus -= 0.5
                moves = self.generate_moves(r, c)
                mobility_black += len(moves)
                if any(move[2] is not None for move in moves):
                    black_forced_capture = True
            elif cell == "b":
                base_score -= 1
                if (r, c) in center_squares:
                    static_center_bonus -= 0.3
                moves = self.generate_moves(r, c)
                mobility_black += len(moves)
                if any(move[2] is not None for move in moves):
                    black_forced_capture = True

    mobility_score = 0.1 * (mobility_white - mobility_black)
    if white_forced_capture:
        forced_capture_bonus += 0.5
    if black_forced_capture:
        forced_capture_bonus -= 0.5
    total_score = base_score + static_center_bonus + mobility_score + forced_capture_bonus

    ''' Penalize Repetitions More Intelligently '''
    for hist_board in self.history:
        if self.board == hist_board:
            total_score -= 200
            break
    if self.history and self.board == self.history[-1]:
        total_score -= 500
    rep_count = sum(1 for hist_board in self.history if hist_board == self.board)
    if rep_count > 1:
        total_score -= 300 * (rep_count - 1)
    if self.history:
        prev_white = sum(cell in ("w", "W") for row in self.history[-1] for cell in row)
        prev_black = sum(cell in ("b", "B") for row in self.history[-1] for cell in row)
        cur_white = sum(cell in ("w", "W") for row in self.board for cell in row)
        cur_black = sum(cell in ("b", "B") for row in self.board for cell in row)
        if cur_white == prev_white and cur_black == prev_black:
            total_score -= 100

    ''' Bonus for material improvement from previous move '''
    if self.history:
        prev_board = self.history[-1]
        prev_white = sum(cell in ("w", "W") for row in prev_board for cell in row)
        prev_black = sum(cell in ("b", "B") for row in prev_board for cell in row)
        cur_white = sum(cell in ("w", "W") for row in self.board for cell in row)
        cur_black = sum(cell in ("b", "B") for row in self.board for cell in row)
        total_score += ((prev_black - cur_black) - (prev_white - cur_white)) * 15

    ''' Bonus for having pieces near the center '''
    dynamic_center_bonus = 0
    center_x, center_y = 3.5, 3.5
    for r in range(8):
        for c in range(8):
            if self.board[r][c] in ("w", "W"):
                dist = abs(r - center_x) + abs(c - center_y)
                dynamic_center_bonus += max(0, 5 - dist) * 0.2
            elif self.board[r][c] in ("b", "B"):
                dist = abs(r - center_x) + abs(c - center_y)
                dynamic_center_bonus -= max(0, 5 - dist) * 0.2
    total_score += dynamic_center_bonus

    ''' Bonus for moving toward promotion and being near promotion '''
    advancement_bonus = 0
    for r in range(8):
        for c in range(8):
            if self.board[r][c] == "w":
                advancement_bonus += (7 - r) * 0.5
            elif self.board[r][c] == "b":
                advancement_bonus += r * 0.5
    promotion_bonus = 0
    for c in range(8):
        if self.board[1][c] == "w":
            promotion_bonus += 2
        if self.board[6][c] == "b":
            promotion_bonus += 2
    total_score += (advancement_bonus + promotion_bonus)

    ''' Pawn Activity Bonus for Black '''
    pawn_activity_bonus = 0
    for r in range(8):
        for c in range(8):
            if self.board[r][c] == "b":
                pawn_activity_bonus += 10
    total_score += pawn_activity_bonus

    ''' Pawn Center Bonus for Black '''
    pawn_center_bonus = 0
    for r in range(8):
        for c in range(8):
            if self.board[r][c] == "b":
                dist = abs(r - center_x) + abs(c - center_y)
                pawn_center_bonus += max(0, 7 - dist) * 2
    total_score += pawn_center_bonus

    ''' Penalty for pieces on the edge '''
    edge_penalty = 0
    for r in range(8):
        for c in range(8):
            if r in (0, 7) or c in (0, 7):
                if self.board[r][c] in ("w", "W", "b", "B"):
                    edge_penalty -= 5
    total_score += edge_penalty

    ''' Aggressive Capture Bonus '''
    capture_bonus = 0
    for r in range(8):
        for c in range(8):
            piece = self.board[r][c]
            moves = self.generate_moves(r, c)
            if piece.lower() == "w":
                for move in moves:
                    if move[2] is not None:
                        capture_bonus += 50
            elif piece.lower() == "b":
                for move in moves:
                    if move[2] is not None:
                        capture_bonus -= 30
    total_score += capture_bonus

    ''' Black King Repetition Penalty '''
    if self.history:
        prev_board = self.history[-1]
        for r in range(8):
            for c in range(8):
                if self.board[r][c] == "B" and prev_board[r][c] == "B":
                    total_score -= 100

    ''' If one side has no pieces, return an extreme value. '''
    cur_white = sum(cell in ("w", "W") for row in self.board for cell in row)
    cur_black = sum(cell in ("b", "B") for row in self.board for cell in row)
    if cur_white == 0:
        return -1000
    if cur_black == 0:
        return 1000

    ''' Endgame Bonus: Increase winning potential when few pieces remain. '''
    total_pieces = sum(cell in ("w", "W", "b", "B") for row in self.board for cell in row)
    if total_pieces <= 6:
        # Amplify material advantage.
        total_score += base_score * 0.5
        # Encourage active king play: if kings are close, favor the side with the material edge.
        white_king_positions = [(r, c) for r in range(8) for c in range(8) if self.board[r][c] == "W"]
        black_king_positions = [(r, c) for r in range(8) for c in range(8) if self.board[r][c] == "B"]
        if white_king_positions and black_king_positions:
            wk = white_king_positions[0]
            bk = black_king_positions[0]
            distance = abs(wk[0] - bk[0]) + abs(wk[1] - bk[1])
            # In the endgame, a shorter distance between kings can help the side with the edge.
            if base_score > 0:
                total_score += (7 - distance) * 20
            elif base_score < 0:
                total_score -= (7 - distance) * 20

    return total_score

CurrentBoard.evaluate = evaluate

In [155]:
#instance of CurrentBoard.
cb = CurrentBoard()
cb.display()


# Check the evaluation score.
cb.evaluate()


   a b c d e f g h
0    .   .   .   .
1  .   .   .   .  
2    .   .   .   .
3  W   .   B   .  
4    .   .   .   .
5  .   .   b   .  
6    .   .   .   .
7  .   .   .   .  



-46.1

In [157]:
cb = CurrentBoard()
cb.display()

print("State of Board:", cb.state_of_board())
print("Evaluation Score:", cb.evaluate())


   a b c d e f g h
0    .   .   .   .
1  .   .   .   .  
2    .   .   .   .
3  W   .   B   .  
4    .   .   .   .
5  .   .   b   .  
6    .   .   .   .
7  .   .   .   .  

State of Board: Unfinished
Evaluation Score: -46.1


#SearchTree

In this implementation of the search tree, I've enforced a maximum depth to prevent infinite searches and filtered out moves that revert to previously visited states to ensure continuous progress. Child nodes are sorted by evaluation scores, prioritizing the most promising moves, and alpha-beta pruning efficiently eliminates branches that won't influence the final decision. Terminal states receive fixed scores, while other leaf nodes are evaluated using a detailed board evaluation function, ensuring every decision is thoroughly analyzed.

In [159]:
def other(player):
    return "w" if player == "b" else "b"

#search tree
class SearchTreeNode:
    def __init__(self, board_instance, playing_as, ply=0, max_depth=20, verbose=False, verbose_threshold=3):
        self.children = [] #possible next moves
        self.value = None #value of the board
        self.value_is_assigned = False
        self.ply_depth = ply #tracks how many moves deep we are in the search
        self.current_board = board_instance
        self.move_for = playing_as
        self.max_depth = max_depth #the limit to how far ahead the tree will search
        self.verbose = verbose #control debugging output
        self.verbose_threshold = verbose_threshold

        # check to see reached maximum depth or if the game is over.
        state = self.current_board.state_of_board()
        if self.ply_depth >= self.max_depth or state != "Unfinished":
            self.value_is_assigned = True
            if state != "Unfinished":
                self.value = 1000 if state == "white" else -1000
            else:
                self.value = self.current_board.evaluate()
        else:
            self.generate_children()

    def generate_children(self):
        possible_boards = self.current_board.all_moves(self.move_for)
        # Filter out moves that would simply revert to the immediate previous board state
        if self.current_board.history:
            parent_board = self.current_board.history[-1]
            possible_boards = [b for b in possible_boards if b.board != parent_board]
        if not possible_boards:
            self.value = self.current_board.evaluate()
        else:
          # Create a new SearchTreeNode for each possible move
            for board in possible_boards:
                next_player = other(self.move_for)
                child = SearchTreeNode(board, next_player, ply=self.ply_depth+1, max_depth=self.max_depth, verbose=self.verbose, verbose_threshold=self.verbose_threshold)
                self.children.append(child)
                 # Sort children so that the most promising moves are examined first.
            self.children.sort(key=lambda child: child.current_board.evaluate(), reverse=(self.move_for=="w"))

    def min_max_value(self):
      if self.value_is_assigned:
        return self.value

      self.children  = sorted(self.children, key = lambda x:x.min_max_value())

      if ((self.ply_depth % 2) == 0):
        # computers move
        self.value = self.children[-1].value
      else:
        #players move
        self.value = self.children[0].value
      self.value_is_assigned = True

      return self.value
        # Implements the alpha-beta pruning algorithm.
        # This recursively evaluates children nodes while cutting back branches that won't affect the outcome
    def alpha_beta(self, alpha=-float("inf"), beta=float("inf")):
        if self.value is not None:
            return self.value
        if self.move_for == "w":
            value = -float("inf")
            for child in self.children:
                value = max(value, child.alpha_beta(alpha, beta))
                alpha = max(alpha, value)
                if beta <= alpha:
                    break
            self.value = value
        else:
            value = float("inf")
            for child in self.children:
                value = min(value, child.alpha_beta(alpha, beta))
                beta = min(beta, value)
                if beta <= alpha:
                    break
            self.value = value
        return self.value


In [160]:
st= SearchTreeNode(cb, "w", max_depth=6, verbose=True)

In [161]:
st.children[1].children[2].children[1].children[2].children[1].value

In [162]:
st.min_max_value()

-274.2

# Play Game function

computer vs computer game

In [164]:
def play_computer_vs_computer():
    current_board = CurrentBoard()
    player1 = "w"
    player2 = "b"
    current_turn = player1
    move_count = 0

    #runs until the game is over or move limit is reached.
    while current_board.state_of_board() == "Unfinished" and move_count < 50:
        print(f"\nMove {move_count}, turn: {current_turn}")
        current_board.display()
        # Build the search tree from the current board state with a given maximum depth.
        search_tree = SearchTreeNode(current_board, current_turn, max_depth=6, verbose=False)
        best_value = search_tree.min_max_value()

        current_board = search_tree.children[-1].current_board
        current_turn = other(current_turn)
        move_count += 1

     #final board and outcome.
    current_board.display()
    final_state = current_board.state_of_board()
    if final_state == "white":
        print("White wins!")
    elif final_state == "black":
        print("Black wins!")
    else:
        print("Game over!")


In [165]:
cb = CurrentBoard()
cb.display()



   a b c d e f g h
0    .   .   .   .
1  .   .   .   .  
2    .   .   .   .
3  W   .   B   .  
4    .   .   .   .
5  .   .   b   .  
6    .   .   .   .
7  .   .   .   .  



In [166]:
if __name__ == "__main__":
    play_computer_vs_computer()



Move 0, turn: w
   a b c d e f g h
0    .   .   .   .
1  .   .   .   .  
2    .   .   .   .
3  W   .   B   .  
4    .   .   .   .
5  .   .   b   .  
6    .   .   .   .
7  .   .   .   .  


Move 1, turn: b
   a b c d e f g h
0    .   .   .   .
1  .   .   .   .  
2    W   .   .   .
3  .   .   B   .  
4    .   .   .   .
5  .   .   b   .  
6    .   .   .   .
7  .   .   .   .  


Move 2, turn: w
   a b c d e f g h
0    .   .   .   .
1  .   .   .   .  
2    W   .   .   .
3  .   .   .   .  
4    .   .   B   .
5  .   .   b   .  
6    .   .   .   .
7  .   .   .   .  


Move 3, turn: b
   a b c d e f g h
0    .   .   .   .
1  .   .   .   .  
2    .   .   .   .
3  .   W   .   .  
4    .   .   B   .
5  .   .   b   .  
6    .   .   .   .
7  .   .   .   .  


Move 4, turn: w
   a b c d e f g h
0    .   .   .   .
1  .   .   .   .  
2    .   .   .   .
3  .   W   .   .  
4    .   .   .   .
5  .   .   b   B  
6    .   .   .   .
7  .   .   .   .  


Move 5, turn: b
   a b c d e f g h
0    .   .   .   .


##Evaluation

This game works and demonstrates the searchtree and evaluation functions well. However, the evaluation function isn't perfect and still has room for improvement. By carefully adjusting its parameters and heuristic values, we could achieve better balance and optimization. Fine-tuning these elements would help the algorithm more accurately capture subtle game dynamics, leading to more strategic, engaging, and realistic gameplay.